In [ ]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

import json
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from research.weight_estimation.keypoint_utils.body_parts import BodyParts
from research.utils.image_utils import Picture
from research.weight_estimation.keypoint_utils.akpr import get_homography_and_matches, generate_sift_adjustment

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
body_parts = BodyParts().get_core_body_parts()
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM keypoint_annotations
    WHERE pen_id=5
    AND captured_at BETWEEN '2019-06-05' AND '2019-07-02'
    AND keypoints is not null
    AND keypoints -> 'leftCrop' is not null
    AND keypoints -> 'rightCrop' is not null
    AND is_qa = TRUE;
"""

mdf = rds_access_utils.extract_from_database(query)
print('Manual dataframe loaded!')

adf = pd.concat([
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-05,2019-06-12).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-12,2019-06-19).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-19,2019-06-26).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-26,2019-07-03).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-07-03,2019-07-04).csv')
])

adf = adf[adf.akpd_score > 0.9].copy(deep=True)
print('AKPD dataframe loaded!')

url_intersection = sorted(list(set(mdf.left_image_url).intersection(adf.left_crop_url)))
df = adf[adf.left_crop_url.isin(url_intersection)].sort_values('left_crop_url')
df['manual_keypoints'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').keypoints.values
df['camera_metadata'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').camera_metadata.values
df['left_crop_metadata'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').left_crop_metadata.values
df['right_crop_metadata'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').right_crop_metadata.values



df = df.sort_values('captured_at')
df['estimated_weight_g'] = df.weight
df.index = pd.to_datetime(df.captured_at)
print('Manual and AKPD Dataframes Joined')

In [ ]:
idx = 0
row = df.iloc[idx]
left_picture = Picture(s3_access_utils=s3_access_utils, image_url=row.left_crop_url)
left_picture.enhance()
left_image = left_picture.get_image()

In [ ]:
left_image_arr = left_picture.get_image_arr()
blurred = cv2.GaussianBlur(left_image_arr, (21, 21), 0)
sharpened = cv2.addWeighted(left_image_arr, 2.0, blurred, -1.0, 0)



In [ ]:
Image.fromarray(cv2.GaussianBlur(sharpened, (3, 3), 0))

In [ ]:
left_image